#This project encompasses a beta project that assists in Research of Stock health per company all over the world

#As a starting step, It currently automates the Team's research on Stock and Crypto in China's GBA
#TODO-Measure the accuracy of the model


In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


In [ ]:
#Baseline_Setup_and_dependencies
#Pegasus Tokenizer, NLP for summarization
#BS4 scraps the web for news
import sentencepiece
import urllib
import requests
import selenium
import pickle
import re
import tensorflow
from transformers import PegasusTokenizer,PegasusForConditionalGeneration, TFPegasusForConditionalGeneration
from bs4 import BeautifulSoup
from transformers import pipeline
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [ ]:
#Loading and saving the summarization models  from the Hugging face model

model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

'''
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")
model = AutoModelForSeq2SeqLM.from_pretrained("human-centered-summarization/financial-summarization-pegasus")
'''

'\nfrom transformers import AutoTokenizer, AutoModelForSeq2SeqLM\ntokenizer = AutoTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")\nmodel = AutoModelForSeq2SeqLM.from_pretrained("human-centered-summarization/financial-summarization-pegasus")\n'

This Code slices focus on the Automation of more than one stock symbol

In [ ]:
#Building a News and Sentiment Pipleline
#Test with company stock names --Edited to user preference
#The model asks which tinkers you wanna monitor and how much
#You can also edit the search URL to use expedia or Bloomberg finance
#Search for stock news using Google and Yahoo Finance
#TODO--> Figure out how to allow a universal Web scrapping instead of relying solely on Yahoo finance. ie investopedia, Bloomberg LP etc

def search_for_stock_news_urls(ticker, source):
  if source == "Bloomberg":
    search_url = f"https://www.google.com/search?q=bloomberg+{ticker}&tbm=nws"
  elif source == "Yahoo":
    search_url = f"https://www.google.com/search?q=yahoo+finance+{ticker}&tbm=nws"
  elif source == "Investopedia":
    search_url = f"https://www.google.com/search?q=investopedia+{ticker}&tbm=nws"
  elif source == "Google Finance":
    search_url = f"https://www.google.com/search?q=google+finance++{ticker}&tbm=nws"
  else:
    return None
  
  r = requests.get(search_url)
  soup = BeautifulSoup(r.text, 'html.parser')
  if source == "Bloomberg":
    atags = soup.find_all('a')
  elif source == "Yahoo":
    atags = soup.find_all('a')
  elif source == "Investopedia":
    atags = soup.find_all('a')
  elif source == "Google Finance":
    atags = soup.find_all('a')
  else:
    return None
  
  hrefs = [link['href'] for link in atags]
  return hrefs

num_tickers = int(input("Enter the number of stock tickers you want to monitor: "))
monitored_tickers = []
for i in range(num_tickers):
  ticker = input(f"Enter the company's stock ticker {i+1}: ")
  monitored_tickers.append(ticker)

sources = ["Bloomberg", "Yahoo", "Investopedia", "Google Finance"]
source_choice = input(f"Enter the source you want to use ({', '.join(sources)}), or type 'all': ")

if source_choice.lower() == "all":
  raw_urls = {}
  for ticker in monitored_tickers:
    raw_urls[ticker] = []
    for source in sources:
      urls = search_for_stock_news_urls(ticker, source)
      if urls is not None:
        raw_urls[ticker].extend(urls)
else:
  raw_urls = {ticker: search_for_stock_news_urls(ticker, source_choice) for ticker in monitored_tickers}

print(raw_urls)


Enter the number of stock tickers you want to monitor: 1
Enter the company's stock ticker 1: AAPL
Enter the source you want to use (Bloomberg, Yahoo, Investopedia, Google Finance), or type 'all': Bloomberg
{'AAPL': ['/?sa=X&ved=0ahUKEwj1pY-jo8f-AhVoElkFHZmPDcQQOwgC', '/search?q=bloomberg+AAPL&tbm=nws&ie=UTF-8&gbv=1&sei=efRIZLWFIuik5NoPmZ-2oAw', '/search?q=bloomberg+AAPL&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwj1pY-jo8f-AhVoElkFHZmPDcQQ_AUIBSgA', '/search?q=bloomberg+AAPL&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwj1pY-jo8f-AhVoElkFHZmPDcQQ_AUIBygC', '/search?q=bloomberg+AAPL&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwj1pY-jo8f-AhVoElkFHZmPDcQQ_AUICCgD', 'https://maps.google.com/maps?q=bloomberg+AAPL&um=1&ie=UTF-8&sa=X&ved=0ahUKEwj1pY-jo8f-AhVoElkFHZmPDcQQ_AUICSgE', '/search?q=bloomberg+AAPL&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwj1pY-jo8f-AhVoElkFHZmPDcQQ_AUICigF', '/search?q=bloomberg+AAPL&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwj1pY-jo8f-AhVoElkFHZmPDcQQ_AUICygG', '/adv

In [ ]:
#Strip out the unwanted URLs--> policies,accounts, preferences etc
excluded_list = ['maps', 'policies', 'preferences', 'support', 'accounts']
def strip_unwanted_urls(urls, excluded_list):
    final_val = []
    for url in urls:
        if 'https://' in url and not any(excluded_word in url for excluded_word in excluded_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            final_val.append(res)
    return list(set(final_val))

cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], excluded_list) for ticker in monitored_tickers}
cleaned_urls


{'AAPL': ['https://www.bloomberg.com/news/articles/2023-04-14/apple-ramps-up-testing-of-new-macbooks-ahead-of-summer-launch',
  'https://www.bloomberg.com/news/articles/2023-04-17/apple-goldman-debut-savings-account-with-4-15-annual-yield',
  'https://www.bloomberg.com/news/articles/2023-04-18/is-apple-s-savings-account-worth-it-4-15-is-high-yield-but-has-downsides',
  'https://www.bloomberg.com/news/articles/2023-04-24/apple-wins-epic-court-battle-while-ceding-ground-on-app-store',
  'https://www.bloomberg.com/news/articles/2023-04-25/apple-aapl-developing-ai-health-coaching-service-ipados-17-health-app',
  'https://www.bloomberg.com/news/articles/2023-04-17/apple-s-india-sales-near-6-billion-as-cook-begins-retail-push',
  'https://www.bloomberg.com/news/articles/2023-04-18/apple-vr-ar-headset-apps-sports-tv-fitness-gaming-wellness-ipad-features',
  'https://www.bloomberg.com/news/newsletters/2023-04-09/apple-aapl-continues-efforts-to-keep-retail-stores-from-unionizing-lg9gjdx2',
  'h

In [ ]:
def scrape_and_process(urls):
    ARTICLES = []
    for url in urls:
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
                'Referer': 'https://www.bloomberg.com/',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
                'Accept-Encoding': 'gzip, deflate, br',
                'Accept-Language': 'en-US,en;q=0.5',
                'Connection': 'keep-alive',
                'Upgrade-Insecure-Requests': '1',
            }
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.content, 'html.parser')
            paragraphs = soup.find_all('p')
            text = [paragraph.text for paragraph in paragraphs]
            words = ''.join(text).split(' ')[:400]
            ARTICLE = ''.join(words)
            ARTICLES.append(ARTICLE)
        except:
            print(f"Error occurred while scraping {url}")
            continue
    return ARTICLES

articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
print(articles)



{'AAPL': ["Tocontinue,pleaseclicktheboxbelowtoletusknowyou'renotarobot.PleasemakesureyourbrowsersupportsJavaScriptandcookiesandthatyouarenot\nblockingthemfromloading.\nFormoreinformationyoucanreviewourTermsof\nServiceandCookiePolicy.Forinquiriesrelatedtothismessagepleasecontact\noursupportteamandprovidethereferenceIDbelow.", "Tocontinue,pleaseclicktheboxbelowtoletusknowyou'renotarobot.PleasemakesureyourbrowsersupportsJavaScriptandcookiesandthatyouarenot\nblockingthemfromloading.\nFormoreinformationyoucanreviewourTermsof\nServiceandCookiePolicy.Forinquiriesrelatedtothismessagepleasecontact\noursupportteamandprovidethereferenceIDbelow.", "Tocontinue,pleaseclicktheboxbelowtoletusknowyou'renotarobot.PleasemakesureyourbrowsersupportsJavaScriptandcookiesandthatyouarenot\nblockingthemfromloading.\nFormoreinformationyoucanreviewourTermsof\nServiceandCookiePolicy.Forinquiriesrelatedtothismessagepleasecontact\noursupportteamandprovidethereferenceIDbelow.", "Tocontinue,pleaseclicktheboxbelowtolet

In [ ]:
#Seach and scrape cleaned URLs

def scrape_and_process(urls):
    ARTICLES = []
    for url in urls:
        try:
            headers = {}
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.content, 'html.parser')
            paragraphs = soup.find_all('p')
            text = [paragraph.text for paragraph in paragraphs]
            words = ''.join(text).split(' ')[:400]
            ARTICLE = ''.join(words)
            ARTICLES.append(ARTICLE)
        except:
            print(f"Error occurred while scraping {url}")
            continue
    return ARTICLES

articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'AAPL': ["Tocontinue,pleaseclicktheboxbelowtoletusknowyou'renotarobot.PleasemakesureyourbrowsersupportsJavaScriptandcookiesandthatyouarenot\nblockingthemfromloading.\nFormoreinformationyoucanreviewourTermsof\nServiceandCookiePolicy.Forinquiriesrelatedtothismessagepleasecontact\noursupportteamandprovidethereferenceIDbelow.",
  "Tocontinue,pleaseclicktheboxbelowtoletusknowyou'renotarobot.PleasemakesureyourbrowsersupportsJavaScriptandcookiesandthatyouarenot\nblockingthemfromloading.\nFormoreinformationyoucanreviewourTermsof\nServiceandCookiePolicy.Forinquiriesrelatedtothismessagepleasecontact\noursupportteamandprovidethereferenceIDbelow.",
  "Tocontinue,pleaseclicktheboxbelowtoletusknowyou'renotarobot.PleasemakesureyourbrowsersupportsJavaScriptandcookiesandthatyouarenot\nblockingthemfromloading.\nFormoreinformationyoucanreviewourTermsof\nServiceandCookiePolicy.Forinquiriesrelatedtothismessagepleasecontact\noursupportteamandprovidethereferenceIDbelow.",
  "Tocontinue,pleaseclicktheboxbelo

In [ ]:
from transformers import pipeline

# Define the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_all_articles(articles):
    summaries = []
    for article in articles:
        # Use the summarizer pipeline to generate a summary
        summary = summarizer(article, max_length=120, min_length=30, do_sample=False)[0]['summary_text']
        summaries.append(summary)
    return summaries

summaries = {ticker:summarize_all_articles(articles[ticker]) for ticker in monitored_tickers}
summaries


Your max_length is set to 120, but you input_length is only 10. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)


{'AAPL': ['GoogleparentAlphabet(NASDAQ:GOOG)(NASDAQ.GOOGL)isdown~30%fromitsall-timehighin2021.Fromits52w-lowof $83.45ithasgained27%,buthasunderperformedboththebroadermarketanditslargetechpeers.',
  'Apple and Googleappstoresshouldhavethesameobligationstochildsafetyascornershops,ithasbeenargued.Parliamentheardyoungsterswereabletoaccessadultproductsofferingcasualsexandgamblingontheplatforms.Thiswaspermittedevenwhenthetechgiants’emphaticallyknow’theuserisachild, accordingtoaToryformerminister.',
  'FAANGisanacronymstandsforthefivemajortechnologycompanieslistedontheUSstockexchange. Inthisarticle,wewilltakealookatthe10bestFAANGstockstobuynow. Ifyouwanttoexploresimilarstocks,youcanalsotakealookat\xa05BestFAANGStocksToBuyNow.',
  "Alphabet'srevenuerose3%to $69.79billionfrom $68billionayearearlier, according to a report. CreditCardsLoansBankingMortgagesInsuranceCreditMonitoringPersonalFinanceSmallBusinessTaxesHelpforLowCreditScoresInvesting.",
  'GoogleCloudislookingto helpWeb3buildersfast-tra

In [ ]:
#Calling the sentiment analysis to perform sentiment labelling for the summaries
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model_revision = "af0f99b"
model = AutoModelForSequenceClassification.from_pretrained(model_name, revision=model_revision)
tokenizer = AutoTokenizer.from_pretrained(model_name, revision=model_revision)
sentiment = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [ ]:
# Adding stop words and phrases
stop_words = ['Ourengineersareworkingquicklytoresolvetheissue. Thankyouforyourpatience. Back to Mail Online home. back to the page you came from.',' PleasemakesureyourbrowsersupportsJavaScriptandcookiesandthatyouarenotblockingthemfromloading. Tocontinue,pleaseclicktheboxbelowtoletusknowyou renotarobot.' , '2023-私隱權政策-條款. ©2023. www.japanewspaper.com']

# Adding Sentiment Analysis to give a Negative/Positive score
# -VE means the market is not good for the product
# +VE means the market is good for the product/investment
scores = {
    ticker: sentiment(summaries[ticker])
    for ticker in monitored_tickers
}
for ticker in scores:
    print(f"\nSentiment analysis for {ticker}:")
    for i, score in enumerate(scores[ticker]):
        summary = summaries[ticker][i]
        if any(word in summary for word in stop_words) or re.match(r'^\s*$', summary):
            continue  # Skip if the sentiment contains stop words or phrases, or is blank
        print(f"{i+1}. Summary: {summary}")
        print(f"   Score: {score['score']}, Label: {score['label']}")
    negative_count = sum(
        1 for score in scores[ticker] if score['label'] == 'NEGATIVE'
    )
    if negative_count > 5:
        print(f"\nThe model finds that stock {ticker} is not doing well currently. We recommend that you don't buy for short holding.")
    elif negative_count == 5:
        print(f"\nThe model generates a neutral view on stock {ticker}, further research is needed: Bloomberg / Expedia refinement.")
    elif negative_count < 5:
        print(f"\nThe model finds that stock {ticker} is good to buy for the short term. Contact our brokers to buy.")



Sentiment analysis for AAPL:
1. Summary: GoogleparentAlphabet(NASDAQ:GOOG)(NASDAQ.GOOGL)isdown~30%fromitsall-timehighin2021.Fromits52w-lowof $83.45ithasgained27%,buthasunderperformedboththebroadermarketanditslargetechpeers.
   Score: 0.9978030323982239, Label: NEGATIVE
2. Summary: Apple and Googleappstoresshouldhavethesameobligationstochildsafetyascornershops,ithasbeenargued.Parliamentheardyoungsterswereabletoaccessadultproductsofferingcasualsexandgamblingontheplatforms.Thiswaspermittedevenwhenthetechgiants’emphaticallyknow’theuserisachild, accordingtoaToryformerminister.
   Score: 0.9914022088050842, Label: NEGATIVE
3. Summary: FAANGisanacronymstandsforthefivemajortechnologycompanieslistedontheUSstockexchange. Inthisarticle,wewilltakealookatthe10bestFAANGstockstobuynow. Ifyouwanttoexploresimilarstocks,youcanalsotakealookat 5BestFAANGStocksToBuyNow.
   Score: 0.9895598888397217, Label: NEGATIVE
4. Summary: Alphabet'srevenuerose3%to $69.79billionfrom $68billionayearearlier, according t

This slice is for exporting a summary

In [ ]:
#Step-1 is to summarize the blueprint labels for the CSV
#show why the previous tinker is not good to buy
#Fix APPL error
def create_output_array(summaries, scores, urls):
  output = []
  for ticker in monitored_tickers:
    for counter in range(len(summaries[ticker])):
      output_this = [
      ticker,
      summaries[ticker][counter],
      scores[ticker][counter],
      urls[ticker][counter],
    ]
    output.append(output_this)
  return output

final_output = create_output_array(summaries,scores,cleaned_urls)
final_output

[['AAPL',
  'Apple isaneobanklikeChime,Revolut andMonzo–except itsbrandstrengthisunparalleledgiventhattherearemorethantwobillioniPhonesglobally.',
  {'label': 'NEGATIVE', 'score': 0.9840976595878601},
  'https://www.forbes.com/sites/emilymason/2023/04/24/why-apples-partnership-with-goldman-is-the-future-of-banking/']]

In [ ]:
#Exporting the results
#TOCO--> Fix the csv output
final_output.insert(0, ['Ticker', 'Summary', 'Label', 'Confidence', 'Url'])
import csv
with open('assetsummaries.csv', mode='w', newline='') as f:
  csv_writer = csv.writer(f, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
  csv_writer.writerows(final_output)